In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/05 16:11
# @Author  : Wang Yujia
# @File    : SA_for_PT_model_unsolved.ipynb
# @Description : Parameter estimation for PT_model using Simulated Annealing and Delta==1.
#                   Some settings fail in inference and I wonder why.

# 0. What for
1. 一些参数infer不出来，这里探究一下why

# 1. Preparations
## 1.1 全局设置

In [148]:
# data_key path
data_key_path = "../data/SA_PT/data_key.csv"
# inferred params
params_opitim_delta_wset_path = "../data/SA_PT/params_opitim_delta_wset.csv"
# 初始参数
table_5_M = [0.025,3.72]

import numpy as np
import pandas as pd
from visdom import Visdom
from SA_for_PT_funcs_delta_eq1 import *

## 1.2 data 读取

In [176]:
data_key = pd.read_csv(data_key_path, encoding="utf-8")
params_opitim_delta_wset = pd.read_csv(params_opitim_delta_wset_path, encoding="utf-8")

# 1.3. 提取没能infer的auction参数

In [191]:
idx_na = pd.isna(params_opitim_delta_wset.alpha)
params_na = params_opitim_delta_wset[idx_na][:].reset_index()
print(f"一共有 *{params_na.shape[0]}* 个auction setting没能infer出来（NaN）")
viz = Visdom(env='002')

Setting up a new session...


一共有 *30* 个auction setting没能infer出来（NaN）


# 1.4 cal u

In [247]:
def cal_u(params,other_params):
    alpha = params[0]
    labda = params[1]
    max_T,v,d,b = other_params

    # solve for U from Equi. condt.
    U_i = [0] * (max_T + 1)
    U_i[0] = 1
    flag = 1
    flag_t = 0
    for t in range(1,max_T+1):

        U_i[t] = f_Equi(t, v, d, b, alpha, labda)
        # 等于或小于0
        #if(flag & (np.isclose(U_i[t],0.0) | (U_i[t]<0))):
        # 小于0
        if(flag & (U_i[t]<0)):
            print(f"t:{t} -- u = 0.0:{U_i[t]}")
            flag_t = t
            flag = 0
    if(U_i[0]==1):
        U_i.pop(0)
    return U_i,(flag_t-1)

# 2. 分析
## 2.1 是因为max_T太大了导致infer不出来吗
1. max_T很小的时候，也有infer不出来的

In [150]:
viz.bar(np.array(params_na.max_T),win = 'Max_T distrb.', env='002',opts= dict(title='Max_T distrb.'))
print(f"max_T 平均值是 *{np.mean(np.array(params_na.max_T))}*")
print(f"max_T 最小的是 *{np.min(np.array(params_na.max_T))}*")

max_T 平均值是 *10089.266666666666*
max_T 最小的是 *107*


## 2.2 infer时，计算初始loss会出问题吗
1. 用`t_u_lt0`记录下每个setting里，第一个<0的t值，如果`t_u_lt0[t] == -1`这说明并没有<0的数
2. 这30个均出现了<=0的情况。而且<0的情况有 21/30的比例，只有9个setting下不存在<0的u

In [248]:
#viz = Visdom(env='003')
t_u_lt0 = []
U_na = pd.DataFrame()
for i in range(0,params_na.shape[0]):

    max_T_i = int(np.array(params_na.max_T)[i])
    v = float(np.array(params_na.retail)[i])
    d = float(np.array(params_na.bidincrement)[i])
    b = float(np.array(params_na.bidfee)[i])

    U_i,flag_t = cal_u(table_5_M,other_params = [max_T_i,v,d,b])
    U_na.append(U_i)
    t_u_lt0.append(flag_t)
    #viz.line(np.array(U_i),np.arange(0,U_i.__len__()),win = 'U of '+str(i),env='003',opts=dict(title='U of '+str(i)))

t:1662 -- u = 0.0:-1.4807183343880737e-06
t:1996 -- u = 0.0:-1.0552703008402242e-06
t:2662 -- u = 0.0:-3.4756774069114724e-08
t:4862 -- u = 0.0:-9.080125649259905e-12
t:3329 -- u = 0.0:-7.115308336219532e-09
t:8330 -- u = 0.0:-5.551115123125932e-17
t:7996 -- u = 0.0:-2.2204460492505226e-16
t:529 -- u = 0.0:-0.0002992513676255309
t:7996 -- u = 0.0:-1.665334536937892e-16
t:529 -- u = 0.0:-0.0002992513676255309
t:529 -- u = 0.0:-0.0002992513676255309
t:246 -- u = 0.0:-0.0012334214791850735
t:1496 -- u = 0.0:-6.902474420174295e-06
t:856 -- u = 0.0:-7.8974108604033e-05
t:856 -- u = 0.0:-7.8974108604033e-05
t:356 -- u = 0.0:-0.0006691220611303275
t:96 -- u = 0.0:-0.004422011803766538
t:246 -- u = 0.0:-0.001368530438202125
t:1496 -- u = 0.0:-1.7080880446999987e-05
t:491 -- u = 0.0:-0.00022321778611344916
t:2991 -- u = 0.0:-9.531666966272318e-08


3. 有3/30个auction，u恒>0；有6/30个auction，u=0而没有<0；有 21/30个auction,出现了u<0的情况
    - 3个auction：降低SA的停止条件可以infer出来
    - 6个auction：

In [246]:
# 只运行一次就好
idx_u_gt0 = []
for i in range(0,params_na.shape[0]):
    if(t_u_lt0[i] == -1):
        idx_u_gt0.append(i)
print(f"在30个没infer出结果的auction setting里，有 *{idx_u_gt0.__len__()}* 个并未出现u<=0的情况（u均>0）")
print(params_na.iloc[idx_u_gt0][:])

在30个没infer出结果的auction setting里，有 *3* 个并未出现u<=0的情况（u均>0）
    index  product_id  bidincrement  bidfee   retail  key_idx  alpha  delta  \
6     144    10010498          0.01    0.75  1899.99      144    NaN    NaN   
17    469    10010868          0.01    0.75   149.99      469    NaN    NaN   
21    540    10011706          0.01    0.75   225.00      540    NaN    NaN   

    labda  initial_loss  final_loss  avg_loss  max_T         T  
6     NaN           NaN         NaN       NaN  18091  189924.0  
17    NaN           NaN         NaN       NaN   2155   14924.0  
21    NaN           NaN         NaN       NaN   7745   22425.0  


In [260]:
# 只运行一次就好
idx_u_eq0 = [10,12,13,18,19,22]
idx_u_ls0 = []
for i in range(0,params_na.shape[0]):
    if (np.isin(i,idx_u_gt0) | np.isin(i,idx_u_eq0)):
        continue
    else:
        idx_u_ls0.append(i)
print(params_na.iloc[idx_u_ls0][:])

    index  product_id  bidincrement  bidfee   retail  key_idx  alpha  delta  \
0      21    10006485          0.15    0.75   249.99       21    NaN    NaN   
1      30    10002407          0.15    0.75   300.00       30    NaN    NaN   
2      54    10010735          0.15    0.75   399.99       54    NaN    NaN   
3      61    10010648          0.15    0.75   729.99       61    NaN    NaN   
4     124    10009874          0.15    0.75   499.95      124    NaN    NaN   
5     133    10010049          0.15    0.75  1249.99      133    NaN    NaN   
7     149    10010794          0.15    0.75  1199.99      149    NaN    NaN   
8     151    10010878          0.15    0.75    79.95      151    NaN    NaN   
9     175    10010871          0.15    0.75  1200.00      175    NaN    NaN   
11    235    10011047          0.15    0.75    79.95      235    NaN    NaN   
14    331    10011308          0.15    0.75    79.95      331    NaN    NaN   
15    416    10009318          0.15    0.75    37.50

## 2.3 可以infer的setting的u是怎么样的呢
3. 但是有一些初始loss为inf（比如max_T较大的情况），但是也可以infer出来的。筛选出这些settings
4. 筛选出`max_T>20000`并且能infer出来结果（至少不会卡住的）的setting `params_2w`，计算他们的初始u

In [80]:
idx_2w = ((pd.notna(params_opitim_delta_wset.alpha)) & (params_opitim_delta_wset.max_T>20000))
params_2w = params_opitim_delta_wset[idx_2w][:]
viz = Visdom(env='004')

Setting up a new session...


In [229]:
t_u_lt0 = []
for i in range(0,params_2w.shape[0]):

    max_T_i = int(np.array(params_2w.max_T)[i])
    v = float(np.array(params_2w.retail)[i])
    d = float(np.array(params_2w.bidincrement)[i])
    b = float(np.array(params_2w.bidfee)[i])

    U_i,flag_t = cal_u(table_5_M,other_params = [max_T_i,v,d,b])
    t_u_lt0.append(flag_t)
    viz.line(np.array(U_i),np.arange(0,U_i.__len__()),win = 'U of '+str(i),env='004',opts=dict(title='U of '+str(i)))

t:6662 -- u = 0.0:-1.0602629885319147e-14
t:6662 -- u = 0.0:-8.881784197126003e-15
t:6662 -- u = 0.0:-8.881784197126003e-15
t:7996 -- u = 0.0:-2.2204460492505226e-16


## 2.4 对于不同的alpha，曲线u与x轴的交点会变化吗
1. 设置了6组参数，分别单独改变alpha的值，观察交点变化，记录在`t_u_lt0_alpha`中
2. **发现这个交点t不会变**，t不会变（个别极限除外）

In [135]:
params_initial = [[-0.025,3.72],[-0.001,3.72],[0.025,3.72],[0.05,3.72],[0.1,3.72],[0.2,3.72]]
t_u_lt0_alpha = []
for j in range(0,params_initial.__len__()):
    table_5_M = params_initial[j]
    t_u_lt0_j = []
    for i in range(0,params_na.shape[0]):

        max_T_i = int(np.array(params_na.max_T)[i])
        v = float(np.array(params_na.retail)[i])
        d = float(np.array(params_na.bidincrement)[i])
        b = float(np.array(params_na.bidfee)[i])

        U_i,flag_t = cal_u(table_5_M,other_params = [max_T_i,v,d,b])
        t_u_lt0_j.append(flag_t)
    t_u_lt0_alpha.append(t_u_lt0_j)

t_u_lt0_alpha = np.array(t_u_lt0_alpha)

t:1662 -- u = 0.0:-0.000744427145778997
t:1996 -- u = 0.0:-0.0018584884728845303
t:2662 -- u = 0.0:-0.0007430081491805548
t:4862 -- u = 0.0:-0.000742974049472756
t:3329 -- u = 0.0:-0.0018574598455916255
t:8329 -- u = 0.0:-0.0013621264523631298
t:7995 -- u = 0.0:-0.0001238283037920323
t:529 -- u = 0.0:-0.002151126411443148
t:7996 -- u = 0.0:-0.0018574528628445303
t:529 -- u = 0.0:-0.002151126411443148
t:529 -- u = 0.0:-0.002151126411443148
t:246 -- u = 0.0:-0.00306785069471228
t:1496 -- u = 0.0:-0.0018642267392213168
t:856 -- u = 0.0:-0.0019349554612040475
t:10329 -- u = 0.0:-0.001362126452364771
t:856 -- u = 0.0:-0.0019349554612040475
t:356 -- u = 0.0:-0.002514095777063806
t:96 -- u = 0.0:-0.005844326857626388
t:246 -- u = 0.0:-0.0028368290389451985
t:1496 -- u = 0.0:-0.001505591809338759
t:491 -- u = 0.0:-0.0009642606811450834
t:2990 -- u = 0.0:-8.636313063562333e-16
t:1662 -- u = 0.0:-0.00013578625579518548
t:1996 -- u = 0.0:-0.0002895119397696306
t:2662 -- u = 0.0:-9.10456052662566e

## 2.5 T和Max_T的差别有多大
1. 首先理论duration的上界普遍偏大，大于实际duration。

In [198]:
viz = Visdom(env='002')
diff_T = np.array(params_opitim_delta_wset['max_T'] - params_opitim_delta_wset['T'])
# print(f"实际duration和理论duration的上界之差，平均有： *{np.mean(diff_T)}*")
# print(f"实际duration和理论duration的上界之差，中位数有： *{np.median(diff_T)}*")
print(f"有 *{sum(diff_T>0) / diff_T.__len__()}* 的auction setting，实际duration > 理论上限")

viz.bar(diff_T,win = 'diff for all: Max_T - T', env='002',opts= dict(title='diff for all: Max_T - T'))

Setting up a new session...


有 *0.1335379892555641* 的auction setting，实际duration > 理论上限


'diff for all: Max_T - T'

2. 而infer不出来的参数，他们的diff则普遍实际大于理论
3. 虽然并不是所有“实际duration > 理论上限”的auction都infer不出来，**但是这个确实是infer不出来的重要原因之一。**
    - 在T>0的时候

In [199]:
diff_T_na = np.array(params_na['max_T'] - params_na['T'])
print("对于infer不出来的30个setting来说")
print(f"有 *{sum(diff_T_na>0) / diff_T_na.__len__()}* 的auction setting，实际duration > 理论上限")
viz.bar(diff_T_na,win = 'diff for uninferred: Max_T - T', env='002',opts= dict(title='diff for uninferred: Max_T - T'))

对于infer不出来的30个setting来说
有 *0.7333333333333333* 的auction setting，实际duration > 理论上限


'diff for uninferred: Max_T - T'

## 2.6 f(x)与alpha和x的关系
$f(x) = 1-exp((-\alpha*x))$
1. 当`alpha<0`时，此时f随着x递减此时f随着x递减, 但是x很大时，`f(x)`无限接近-inf ：
> RuntimeWarning: overflow encountered in exp return (1-np.exp(-alpha*x))
2. 当`alpha>0`时，f随着x递增直到1然后不变。

In [225]:
x = np.arange(0,1000)
viz.line(f(x,0.025),x,win = 'f_alpha_gt0', env='002',opts= dict(title='f_alpha_gt0'))
# 会提示溢出
viz.line(f(x,-0.025),x,win = 'f_alpha_ls0', env='002',opts= dict(title='f_alpha_ls0'))

'f_alpha_ls0'

3. 比较一下f的导数和alpha的关系：
    - $df = (-\alpha)*exp((-\alpha*x))$
    - `alpha>0`时，导数有界而且，当x比较大时，导数增速也很慢
    - `alpha<0`时，导数是一个指数函数，绝对值可以到inf

In [226]:
viz.line(0.025*f(x,0.025),x,win = 'df_alpha_gt0', env='002',opts= dict(title='df_alpha_gt0'))
viz.line((-0.025)*f(x,-0.025),x,win = 'df_alpha_ls0', env='002',opts= dict(title='df_alpha_ls0'))

'df_alpha_ls0'

# 3
## 3.1 对于Max_T<5000的重新取一下初始值去infer, 看看u的情况

In [93]:
idx_5k = ((pd.isna(params_na.alpha)) & (params_na.max_T<5000))
params_5k = params_na[idx_5k][:]
print(f"一共有 *{params_5k.shape[0]}* 个auction setting需要在新的初始值下infer")
viz = Visdom(env='003')

Setting up a new session...


一共有 *15* 个auction setting需要在新的初始值下infer


In [99]:
for i in range(0,params_5k.shape[0]):

    max_T_i = int(np.array(params_5k.max_T)[i])
    v = float(np.array(params_5k.retail)[i])
    d = float(np.array(params_5k.bidincrement)[i])
    b = float(np.array(params_5k.bidfee)[i])

    U_i,flag_t = cal_u(table_5_M,other_params = [max_T_i,v,d,b])
    t_u_lt0.append(flag_t)
    viz.line(np.array(U_i),np.arange(0,U_i.__len__()),win = 'U‘ of '+str(i),env='003',opts=dict(title='U’ of '+str(i)))



t:529 -- u = 0.0:-1.9515001420767365e-09
t:529 -- u = 0.0:-1.9515001420767365e-09
t:529 -- u = 0.0:-1.9515001420767365e-09
t:246 -- u = 0.0:-9.50132721389005e-06
t:856 -- u = 0.0:-1.0713652187703878e-13
t:856 -- u = 0.0:-1.0713652187703878e-13
t:356 -- u = 0.0:-3.502404641295848e-07
t:96 -- u = 0.0:-0.0013405360598030122
t:246 -- u = 0.0:-3.3224679378562344e-05
t:1505 -- u = 0.0:-5.551115123125784e-17
t:491 -- u = 0.0:-4.658522124586175e-08


D:\Anaconda\envs\pythorch\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


inf